In [41]:
import numpy as np
import pandas as pd
import pyarrow

from pandas_profiling import ProfileReport

from ipyleaflet import * 
from ipyleaflet import (
    Map, TileLayer, SplitMapControl, Marker, basemaps, MarkerCluster, Choropleth
)

import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import holidays
from datetime import timedelta
from sklearn.inspection import permutation_importance

us_ca_holidays = holidays.UnitedStates(state="CA", years=range(2010, 2025))


In [2]:
#df = pd.read_parquet('./data/safety-SanFrancisco.parquet') 
df = pd.read_parquet('./data/safety-SanFrancisco-2.parquet') 
df = df.drop(['dataType', 'extendedProperties'], axis=1)
for col in ["dataSubtype", "category", "subcategory"]:
    df[col] = pd.Categorical(df[col])
df['dateTime'] = pd.to_datetime(df['dateTime'], format='%Y%m%d %H:%M:%S')

df = pd.DataFrame.drop_duplicates(df)

top_categories = pd.Series(['Street and Sidewalk Cleaning'
                            #'Potentially Life-Threatening'
                            #,'Graffiti'
                            #,'Encampments'
                            #,'Street and Sidewalk Cleaning'
                            #,'Non Life-threatening'
                           ])
df = df[df['category'].isin(top_categories)]

mask = (
    (df["dateTime"] >= pd.to_datetime("2015-01-01")) & (df["dateTime"] < pd.to_datetime("2019-12-31"))
)

df = df[mask]
#df.index=df['dateTime']

#df['DayOfWeek']=df["dateTime"].dt.day_name()
df['DayOfWeek']=df["dateTime"].dt.dayofweek
df['Hour']=df["dateTime"].dt.hour
df['Month']=df["dateTime"].dt.month
df['Day']=df["dateTime"].dt.day
df['Year']=df["dateTime"].dt.year
df['DayOfYear']=df["dateTime"].dt.dayofyear

holiday_eve = []
for date in us_ca_holidays.keys():
    holiday_eve.append(date - timedelta(days=1))
    
df['is_holiday']=df["dateTime"].dt.date.isin(holiday_eve)
df.head()

,requestId,dataSubtype,dateTime,category,subcategory,status,address,latitude,longitude,source,neighborhood,DayOfWeek,Hour,Month,Day,Year,DayOfYear,is_holiday
1,2,311_All,2018-07-08 15:00:27,Street and Sidewalk Cleaning,Bulky Items,Closed,"1536 SACRAMENTO ST, SAN FRANCISCO, CA, 94109",37.791887,-122.418188,Mobile/Open311,Nob Hill,6,15,7,8,2018,189,False
10,30,311_All,2019-01-02 17:35:00,Street and Sidewalk Cleaning,City_garbage_can_overflowing,Closed,Intersection of BEACH ST and HYDE ST,37.806656,-122.420563,Phone,Nob Hill,2,17,1,2,2019,2,False
15,43,311_All,2019-07-23 12:04:00,Street and Sidewalk Cleaning,General Cleaning,Closed,"180 PENNSYLVANIA AVE, SAN FRANCISCO, CA, 94107",37.764300,-122.393714,Mobile/Open311,South Beach,1,12,7,23,2019,204,False
22,57,311_All,2018-06-27 10:55:00,Street and Sidewalk Cleaning,General Cleaning,Closed,"850 LEAVENWORTH ST, SAN FRANCISCO, CA, 94109",37.788984,-122.415220,Mobile/Open311,Nob Hill,2,10,6,27,2018,178,False
27,63,311_All,2019-01-17 09:45:00,Street and Sidewalk Cleaning,General Cleaning,Closed,"3305 FOLSOM ST, SAN FRANCISCO, CA, 94110",37.744892,-122.413170,Phone,Mission,3,9,1,17,2019,17,False


In [3]:
df_weather = pd.read_parquet("./data/weather-data-SF-Nyc-Seattle-2015-2020.parquet")
df_weather["date"] = pd.to_datetime(df_weather["date"], format="%Y%m%d")

df_weather = df_weather[df_weather["station"] == "USW00023234"]
df_weather.head()
df_weather.shape
#profile = ProfileReport(df_weather, title="Pandas Profiling Report")
#profile



(12641, 4)

In [4]:
df_weather = df_weather.set_index(["date", "field"])["value"].unstack(-1)
df_weather = df_weather.fillna(method="ffill").fillna(method="bfill")

df_weather.head()


field,AWND,PRCP,SNWD,TAVG,TMAX,TMIN,WSF2
date,,,,,,,
2015-01-01,38.0,0.0,0.0,97.0,128.0,44.0,89.0
2015-01-02,7.0,0.0,0.0,77.0,111.0,33.0,31.0
2015-01-03,7.0,0.0,0.0,77.0,122.0,33.0,31.0
2015-01-04,9.0,0.0,0.0,86.0,128.0,50.0,63.0
2015-01-05,11.0,0.0,0.0,98.0,150.0,61.0,31.0


In [5]:
df["date"] = df["dateTime"].dt.date

df_weather["date_merge"] = df_weather.index.date
    
df2 = df.merge(df_weather, left_on="date", right_on="date_merge", how="left")
df2.shape

del df2["date_merge"]


In [6]:
df = df2

In [7]:
df.tail()

,requestId,dataSubtype,dateTime,category,subcategory,status,address,latitude,longitude,source,...,DayOfYear,is_holiday,date,AWND,PRCP,SNWD,TAVG,TMAX,TMIN,WSF2
826093,1203095,311_All,2017-09-28 12:33:00,Street and Sidewalk Cleaning,Bulky Items,Closed,Intersection of CLAYTON ST and CORBETT AVE,37.758720,-122.446022,Phone,...,271,False,2017-09-28,52.0,0.0,0.0,208.0,272.0,144.0,134.0
826094,1203128,311_All,2017-02-21 10:04:45,Street and Sidewalk Cleaning,General Cleaning,Closed,Intersection of PACIFIC AVE and TRENTON ST,37.796595,-122.409287,Mobile/Open311,...,52,False,2017-02-21,53.0,74.0,0.0,141.0,161.0,100.0,143.0
826095,1203161,311_All,2018-12-10 21:33:49,Street and Sidewalk Cleaning,Bulky Items,Closed,"1024 JACKSON ST, SAN FRANCISCO, CA, 94133",37.795492,-122.412033,Mobile/Open311,...,344,False,2018-12-10,16.0,5.0,0.0,102.0,144.0,78.0,63.0
826096,1203168,311_All,2017-08-20 09:53:26,Street and Sidewalk Cleaning,Bulky Items,Closed,Intersection of WOOLSEY ST and HAMILTON ST,37.723687,-122.408529,Phone,...,232,False,2017-08-20,43.0,0.0,0.0,189.0,222.0,161.0,81.0
826097,1203173,311_All,2019-03-19 14:14:18,Street and Sidewalk Cleaning,Bulky Items,Closed,"1293 WEBSTER ST, SAN FRANCISCO, CA, 94115",37.782656,-122.431373,Web,...,78,False,2019-03-19,31.0,0.0,0.0,133.0,206.0,94.0,125.0


In [8]:
#df = df.drop(['requestId', 'dataSubtype', 'source', 'latitude', 'longitude', 'address', 'status', 'subcategory'], axis=1)

#df = df.drop(['Month', 'Hour', 'Day', 'DayOfWeek', 'requestId', 'dataSubtype', 'source', 'latitude', 'longitude', 'address', 'status', 'subcategory'], axis=1)
#df.index=df['dateTime']
#df.drop(['dateTime'])

top_neighborhood = pd.Series(['Mission'
                            ,'SoMa'
                            ,'Tenderloin'
                            ,'Nob Hill'
                            ,'Ingleside'
                           ])

df = df[df['neighborhood'].isin(top_neighborhood)]

agg_func = {'neighborhood': ['count']}

#agg = df.groupby([pd.Grouper(key='dateTime', freq='4h'), 'neighborhood']).agg(agg_func).reset_index()

agg = df.groupby([pd.Grouper(key='dateTime', freq='4h'), 
                  'neighborhood', 'Year', 
                  'Month', 'Hour', 'Day', 
                  'DayOfWeek', 'DayOfYear', 
                  'AWND','PRCP','SNWD','TAVG',
                  'TMAX','TMIN','WSF2'])["requestId"].count().reset_index().rename(columns={"requestId": "count"})
  
    
#df.groupby(["dateTime", "neighborhood"]).agg(agg_func).head()
#agg.reset_index()
agg.sort_values("dateTime")


,dateTime,neighborhood,Year,Month,Hour,Day,DayOfWeek,DayOfYear,AWND,PRCP,SNWD,TAVG,TMAX,TMIN,WSF2,count
0,2016-01-01 00:00:00,Nob Hill,2016,1,1,1,4,1,39.0,0.0,0.0,70.0,94.0,28.0,76.0,1
1,2016-01-01 04:00:00,Mission,2016,1,4,1,4,1,39.0,0.0,0.0,70.0,94.0,28.0,76.0,2
13,2016-01-01 08:00:00,Tenderloin,2016,1,11,1,4,1,39.0,0.0,0.0,70.0,94.0,28.0,76.0,1
12,2016-01-01 08:00:00,Tenderloin,2016,1,9,1,4,1,39.0,0.0,0.0,70.0,94.0,28.0,76.0,1
11,2016-01-01 08:00:00,SoMa,2016,1,11,1,4,1,39.0,0.0,0.0,70.0,94.0,28.0,76.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105426,2019-12-30 20:00:00,Mission,2019,12,23,30,0,364,34.0,0.0,0.0,110.0,150.0,83.0,81.0,1
105427,2019-12-30 20:00:00,Nob Hill,2019,12,21,30,0,364,34.0,0.0,0.0,110.0,150.0,83.0,81.0,2
105428,2019-12-30 20:00:00,SoMa,2019,12,20,30,0,364,34.0,0.0,0.0,110.0,150.0,83.0,81.0,1
105429,2019-12-30 20:00:00,SoMa,2019,12,23,30,0,364,34.0,0.0,0.0,110.0,150.0,83.0,81.0,1


In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer, StandardScaler
import sklearn
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(
    agg.sort_values("dateTime"), test_size=0.15, shuffle=False
)



In [10]:
df_train.tail()

,dateTime,neighborhood,Year,Month,Hour,Day,DayOfWeek,DayOfYear,AWND,PRCP,SNWD,TAVG,TMAX,TMIN,WSF2,count
89617,2019-06-11 16:00:00,SoMa,2019,6,16,11,1,162,37.0,0.0,0.0,261.0,367.0,194.0,98.0,5
89620,2019-06-11 16:00:00,SoMa,2019,6,19,11,1,162,37.0,0.0,0.0,261.0,367.0,194.0,98.0,3
89616,2019-06-11 16:00:00,Nob Hill,2019,6,19,11,1,162,37.0,0.0,0.0,261.0,367.0,194.0,98.0,2
89612,2019-06-11 16:00:00,Mission,2019,6,18,11,1,162,37.0,0.0,0.0,261.0,367.0,194.0,98.0,16
89614,2019-06-11 16:00:00,Nob Hill,2019,6,17,11,1,162,37.0,0.0,0.0,261.0,367.0,194.0,98.0,4


In [38]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer, StandardScaler
import sklearn
from sklearn.linear_model import Ridge
from sklearn.preprocessing import OrdinalEncoder

column_transformer = ColumnTransformer(
    [
        (
            "cat", OrdinalEncoder(), ["neighborhood"],
        ),
        (
            "passthrough",
            "passthrough",
            ["Month", "Day", "DayOfWeek", "Hour", "DayOfYear", "Year", "AWND", "PRCP", "TAVG", "TMAX", "TMIN", "WSF2"],
        ),
    ]
)

def passthrugh(df):
    return df

pipe = make_pipeline(FunctionTransformer(passthrugh))
X_train = pipe.fit_transform(df_train)
X_test = pipe.transform(df_test)


#est = Ridge().fit(X_train, df_train["count"])

In [30]:
from sklearn.metrics import classification_report

df_test["predicted"] = est.predict(X_test)

est.score(X_test, df_test["count"])

del df_test["predicted"]

In [48]:
from sklearn.experimental import enable_hist_gradient_boosting 
from sklearn.compose import TransformedTargetRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import GradientBoostingRegressor

column_transformer = ColumnTransformer(
    [
        (
            "cat", OrdinalEncoder(), ["neighborhood"],
        ),
        (
            "passthrough",
            "passthrough",
            ["Month", "Day", "DayOfWeek", "Hour", "DayOfYear", "Year", "AWND", "PRCP", "TAVG", "TMAX", "TMIN", "WSF2"],
        ),
    ]
)

est2 = GradientBoostingRegressor()
pipeline = make_pipeline(column_transformer, est2)
#X_train = pipeline.fit_transform(df_train)
#X_test = pipeline.transform(df_test)


#est2 = HistGradientBoostingRegressor().fit(X_train, df_train["count"],)
pipeline.fit(
    X_train, df_train["count"],
)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('cat', OrdinalEncoder(),
                                                  ['neighborhood']),
                                                 ('passthrough', 'passthrough',
                                                  ['Month', 'Day', 'DayOfWeek',
                                                   'Hour', 'DayOfYear', 'Year',
                                                   'AWND', 'PRCP', 'TAVG',
                                                   'TMAX', 'TMIN', 'WSF2'])])),
                ('gradientboostingregressor', GradientBoostingRegressor())])

In [57]:
import eli5

pr_imp = permutation_importance(
    pipeline, X_test, df_test["count"], scoring="neg_root_mean_squared_error"
)

#eli5.show_weights(est2)
eli5.explain_weights(est2, feature_names=['neighborhood', 'Year', 'Month', 'Hour', 'Day', 'DayOfWeek',
       'DayOfYear', 'AWND', 'PRCP', 'TAVG', 'TMAX', 'TMIN', 'WSF2'])


Weight,Feature
0.6093 ± 0.5325,Day
0.2429 ± 0.5379,neighborhood
0.0762 ± 0.2281,Hour
0.0282 ± 0.1863,DayOfYear
0.0144 ± 0.2187,DayOfWeek
0.0105 ± 0.1506,PRCP
0.0050 ± 0.1406,Month
0.0048 ± 0.0969,AWND
0.0025 ± 0.0353,Year
0.0022 ± 0.0553,TMAX
